# 1. imports

In [100]:
import pickle
from vizier.vizier import Vizier

# 2. Submitting HITs

In [107]:
from mturk_choi import *

static_params = {
    'Title': "Draw pic from video - pilot 0 sandbox1",
    'Description': "Viconary is a challenge where the player uses the provided icons to draw the sequence of the main activities shown in the video. (1-4 activities)",
    'Keywords': ", ",
    'frame_height': 1170,
    'Reward': f'{0.15}',
    'AssignmentDurationInSeconds': 3600 * 1,
    'AutoApprovalDelayInSeconds': 3600 * 24 * 3,
    'LifetimeInSeconds': 3600 * 24 * 1,
    'MaxAssignments': 3,
}

template_params = {
    'template_file': 'vid2pic_template.html', 
    'template_dir': 'hit_templates' 
}

client_params = {
    'aws_access_key_id': access_key,
    'aws_secret_access_key': access_secret_key, 
    'profile_name': None,
    'in_production': False,
    'n_threads': 1,
    's3_base_path': '',
}
vizier = Vizier(**client_params)

Account balance is: $10000.00


In [108]:
def gen_task_template_args(vid):
    return {'video_id': vid}

In [109]:
train_vids_filename = 'Charades_v1_train_crops_urls.pickle'
test_vids_filename = 'Charades_v1_test_crops_urls.pickle'
with open(train_vids_filename, 'rb') as train_urls:
    train_vids = pickle.load(train_urls)
    train_vids = list(train_vids.values())
with open(test_vids_filename, 'rb') as test_urls:
    test_vids = pickle.load(test_urls)
    test_vids = list(test_vids.values())

vids_to_submit = train_vids[0:10]# + test_vids

vizier.preview_hit_interface(template_params, **gen_task_template_args(vids_to_submit))

In [110]:
_ = vizier.expected_cost(vids_to_submit, basic_hit_params=static_params)

Batch will cost $5.40


In [111]:
vizier.preview_hit_interface(template_params, **gen_task_template_args(vids_to_submit[0]))

In [112]:
created_hits = vizier.create_hit_group(vids_to_submit, gen_task_template_args, 
                                       basic_hit_params=static_params, template_params=template_params)

Batch will cost $5.40


# 3. Retreive results

In [87]:
# client_params = {
#     'aws_access_key_id': mturk_ai2.access_key,
#     'aws_secret_access_key': mturk_ai2.access_secret_key, 
#     'profile_name': None,
#     'in_production': True,
#     'n_threads': 1,
#     's3_base_path': '',
# }
# vizier_retriever = Vizier(**client_params)

Account balance is: $2.71


In [117]:
submitted_hits = created_hits
# submitted_hits = vizier_retriever.unpickle_this('./submitted_batch_sandbox_20_wed_may_9_13_08_54_2018.pkl')
hit_batch = [h['HIT'] for h in submitted_hits if h]
# hit_batch = all_sbx_hits

In [89]:
# vizier_retriever.expire_hits(hit_batch)

In [90]:
%%time
latest_res = vizier_retriever.get_assignments(hit_batch)

CPU times: user 407 ms, sys: 20.5 ms, total: 428 ms
Wall time: 18.7 s


In [91]:
# assignment_batch = [a for r in latest_res for a in r['Assignments']]
# len(assignment_batch)

140

In [94]:
# vizier_retriever.pickle_this(assignment_batch, 'second_qual_test_games')

# 4. approve

In [96]:
client_params = {
    'aws_access_key_id': mturk_ai2.access_key,
    'aws_secret_access_key': mturk_ai2.access_secret_key, 
    'profile_name': None,
    'in_production': False,
    'n_threads': 1,
    's3_base_path': '',
}
vizier_retriever = Vizier(**client_params)

Account balance is: $10000.00


In [138]:
%%time
all_sbx_hits = vizier_retriever.get_all_hits()

CPU times: user 6.11 ms, sys: 1.16 ms, total: 7.27 ms
Wall time: 1.41 s


In [139]:
len(all_sbx_hits)

2

In [137]:
dels_resp = vizier_retriever.force_delete_hits(all_sbx_hits)

An error occurred (RequestError) when calling the DeleteHIT operation: This HIT is currently in the state 'Unassignable'.  This operation can be called with a status of: Reviewing, Reviewable (1527620975258 s)
An error occurred (RequestError) when calling the DeleteHIT operation: This HIT is currently in the state 'Unassignable'.  This operation can be called with a status of: Reviewing, Reviewable (1527620975443 s)


In [135]:
# _ = vizier_retriever.set_hits_reviewing(all_sbx_hits)

In [136]:
_ = vizier_retriever.approve_hits(all_sbx_hits)

In [105]:
# vizier.pickle_this(all_hits, 'all_prod_hits_5_21.pkl')

In [137]:
# vizier_retriever.delete_hits(all_sbx_hits)

In [20]:
trial_hit_batch = [h for h in all_hits if h['Title'] == static_params['Title']]

In [21]:
len(trial_hit_batch)

400

In [73]:
# mt_client.approve_assignments()